In [1]:
import sys
sys.path.append("/mnt/home/tf_transformers2/")

In [64]:
from tf_transformers.tokenizer import FullSentencePieceTokenizer
from tf_transformers.utils import TFProcessor

In [22]:
tokenizer = FullSentencePieceTokenizer(sp_model_file='/mnt/home/PRE_MODELS/tf_transformers_models/albert_tfhub/assets/30k-clean.model')

In [48]:
# Read SNIPS data

import pandas as pd
import numpy as np
from pathlib import Path
def parse_line(line):
    utterance_data, intent_label = line.split(" <=> ")
    items = utterance_data.split()
    words = [item.rsplit(":", 1)[0]for item in items]
    word_labels = [item.rsplit(":", 1)[1]for item in items]
    return {
        "intent_label": intent_label,
        "words": " ".join(words),
        "word_labels": " ".join(word_labels),
        "length": len(words),
    }

lines_train = Path("/mnt/home/Projects/BERT_slot_filling/albert_new/albert_nlu/train").read_text().strip().splitlines()
parsed = [parse_line(line) for line in lines_train]

df_train = pd.DataFrame([p for p in parsed if p is not None])


lines_valid = Path("/mnt/home/Projects/BERT_slot_filling/albert_new/albert_nlu/valid").read_text().strip().splitlines()
lines_test = Path("/mnt/home/Projects/BERT_slot_filling/albert_new/albert_nlu/test").read_text().strip().splitlines()

df_valid = pd.DataFrame([parse_line(line) for line in lines_valid])
df_test = pd.DataFrame([parse_line(line) for line in lines_test])

# Slot labels
slot_names = ["[PAD]", "[EXTRA]"]
slot_names += Path("/mnt/home/Projects/BERT_slot_filling/albert_new/albert_nlu/vocab.slot").read_text().strip().splitlines()
slot_map = {}
for label in slot_names:
    slot_map[label] = len(slot_map)

In [3]:
df_train

,intent_label,words,word_labels,length
0,AddToPlaylist,Add Don and Sherri to my Meditate to Sounds of...,O B-entity_name I-entity_name I-entity_name O ...,12
1,AddToPlaylist,put United Abominations onto my rare groove pl...,O B-entity_name I-entity_name O B-playlist_own...,8
2,AddToPlaylist,add the tune by misato watanabe to the Trapeo ...,O O B-music_item O B-artist I-artist O O B-pla...,10
3,AddToPlaylist,add this artist to my this is miguel bosé play...,O O B-music_item O B-playlist_owner B-playlist...,10
4,AddToPlaylist,add heresy and the hotel choir to the evening ...,O B-entity_name I-entity_name I-entity_name I-...,11
...,...,...,...,...
13079,SearchScreeningEvent,find a Consolidated Theatres showing The Good ...,O O B-location_name I-location_name O B-movie_...,10
13080,SearchScreeningEvent,where can i see animated movies in the neighbo...,O O O O B-movie_type I-movie_type B-spatial_re...,9
13081,SearchScreeningEvent,Showtimes for animated movies in the area .,O O B-movie_type I-movie_type B-spatial_relati...,8
13082,SearchScreeningEvent,Which animated movies are playing at Megaplex ...,O B-movie_type I-movie_type O O O B-location_n...,11


In [4]:
# For NER processinf, we expect (sentence, label) both as sentence
# We do a assert(len(sentence.split()), len(label.split())) 
# If this assertion fails, you cant proceed

In [124]:
SPECIAL_PIECE = '▁'

def create_tag_data(df):
    for index, row in df.iterrows():
        sentence = row['words']
        sentence_label = row['word_labels']

        assert(len(sentence.split()) == len(sentence_label.split()))
        words_tokens = sentence.split()
        label_tokens = sentence_label.split()
        subwords = tokenizer.tokenize(sentence)

        # Convert text into main_words (list of list of subwords per word)
        main_words  = []
        temp_tokens = []
        for tok in subwords:
            if tok == SPECIAL_PIECE:
                if temp_tokens:
                    main_words.append(temp_tokens)
                    temp_tokens = []
                main_words.append([tok])

            else:
                if tok.startswith(SPECIAL_PIECE):
                    if temp_tokens:
                        main_words.append(temp_tokens)
                        temp_tokens = []
                    temp_tokens.append(tok)
                else:
                    temp_tokens.append(tok)

        if temp_tokens:
            main_words.append(temp_tokens)

        special_counter = 0
        aligned_words  = []
        aligned_labels = []
        for index,w in enumerate(main_words):
            if len(w) == 1 and w[0] == SPECIAL_PIECE:
                special_counter += 1
                aligned_words.append(w[0])
                aligned_labels.append('[EXTRA]')
            else:
                pos = index - special_counter
                aligned_words.append(words_tokens[pos])
                aligned_labels.append(label_tokens[pos])

        # for (aw, mw, al) in zip(aligned_words, main_words, aligned_labels):
        #     print(aw, '-->', mw, '-->', al)

        flat_tokens = []
        flat_labels = []

        # The first word of the subword token is assigned entity
        # other tokens will be add PAD labels (we will mask it while training)
        for (aw, mw, al) in zip(aligned_words, main_words, aligned_labels):
            temp_w = []
            for w in mw:
                temp_w.append(w)
            temp_l = ['[PAD]'] * len(temp_w)
            temp_l[0] = al
            flat_tokens.extend(temp_w)
            flat_labels.extend(temp_l)
        flat_tokens = ['[CLS]'] + flat_tokens + ['[SEP]']
        flat_labels = ['[PAD]'] + flat_labels + ['[PAD]']

        input_ids  = tokenizer.convert_tokens_to_ids(flat_tokens)
        input_type_ids = [0] * len(input_ids)
        input_mask = [1] * len(input_ids)
        
        label_mask = []
        for idx in range(len(flat_labels)):
            if flat_labels[idx] in ['[PAD]']:
                label_mask.append(0)
                continue
            elif flat_tokens[idx] in ['[CLS]', '[SEP]']:
                label_mask.append(0)
                continue
            else:
                label_mask.append(1)

        token_labels = [slot_map[label] for label in flat_labels]
        assert(len(input_ids) == len(input_mask) == len(input_type_ids) == len(token_labels))
        result = {}
        result['input_ids'] = input_ids
        result['input_type_ids'] = input_type_ids
        result['input_mask'] = input_mask
        result['token_labels'] = token_labels
        result['labels_mask']  = label_mask
        
        yield result

In [125]:
tfprocessor = TFProcessor()
dataset = tfprocessor.process(parse_fn=create_tag_data(df_train))

INFO:absl:Total individual observations/examples written is 13084


In [ ]:
tfprocessor = TFProcessor()
valid_dataset = tfprocessor.process(parse_fn=create_tag_data(df_valid))

In [126]:
def ragged_dict_to_tensor(x):
    x_temp = {}
    for k,v in x.items():
        x_temp[k] = v.to_tensor()
    return x_temp

def separate_x_y(input_dict):
    x = {}
    y = {}
    for k,v in input_dict.items():
        if k in ['input_ids', 'input_mask', 'input_type_ids']:
            x[k] = v
        else:
            y[k] = v
    return (x, y)

batch_size = 25
dataset = dataset.batch(batch_size)
dataset = dataset.map(ragged_dict_to_tensor)
dataset = dataset.map(separate_x_y)

EPOCHS = 3
dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

dataset = dataset.repeat(EPOCHS + 1)

In [127]:
for item in dataset:
    print(item)
    break

({'input_ids': <tf.Tensor: shape=(5, 31), dtype=int32, numpy=
array([[    2,    13,     1,  8096,    13,     1,   218,    17,    13,
            1,  1694,   928,    20,    51,    13,     1,    69, 22443,
           20,    13,     1, 13294,    43,    18,    16,    13,     1,
           58,  6418, 27063,     3],
       [    2,   442,    13,     1, 15424,    43,    13,     1,  1192,
        15971,    18,  1204,    51,  2890, 12489, 27063,     3,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [    2,  3547,    14,  6768,    34,  2462,  6043, 29592,    20,
           14,    13,     1, 10817,  3894, 27063,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [    2,  3547,    48,  1169,    20,    51,    48,    25,  8025,
        11054,     1, 27063,     3,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,    

In [148]:
# Lets load Albert Model
from transformers import TFAlbertModel
from tf_transformers.models import AlbertEncoder

import tensorflow as tf
import json

from tf_transformers.utils import convert_albert_hf_to_tf_transformers
from tf_transformers.core import optimization

from absl import logging
logging.set_verbosity("INFO")

In [89]:
# Load HF model

# Always do this
tf.keras.backend.clear_session()

model_hf_location = '/mnt/home/PRE_MODELS/HuggingFace_models/albert-base-v2/'
model_hf = TFAlbertModel.from_pretrained(model_hf_location)

# Load tf_transformers model
# Most config we will be providing

# Default configs for the model
config_location = '/mnt/home/tf_transformers2/tf_transformers/model_directory/albert_base_v2/albert_config.json'
config = json.load(open(config_location))

# Always do this
tf.keras.backend.clear_session()

# tf_transformers Layer (an extension of Keras Layer)
# This is not Keras model, but extension of keras Layer


model_layer = AlbertEncoder(config=config,
                      name='albert',
                      mask_mode=config['mask_mode'],
                      is_training=False, 
                      use_dropout=False,
                      )
# model_dir = None, because we have not initialized the model with proper variable values
model_tf_transformers = model_layer.get_and_load_model(model_dir=None)

# Convert HF to tf_transformers model
convert_albert_hf_to_tf_transformers(model_hf, model_tf_transformers, config)

All model checkpoint layers were used when initializing TFAlbertModel.

All the layers of TFAlbertModel were initialized from the model checkpoint at /mnt/home/PRE_MODELS/HuggingFace_models/albert-base-v2/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.
INFO:absl:We are overwriding `is_training` is False to `is_training` to True with `use_dropout` is False, no effects on your inference pipeline
INFO:absl:Inputs -->
INFO:absl:input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:input_mask ---> Tensor("input_mask:0", shape=(None, None), dtype=int32)
INFO:absl:input_type_ids ---> Tensor("input_type_ids:0", shape=(None, None), dtype=int32)
INFO:absl:Initialized Variables
INFO:absl:Inputs -->
INFO:absl:input_ids ---> Tensor("input_ids_1:0", shape=(None, None), dtype=int32)
INFO:absl:input_mask ---> Tensor("input_mask_1:0", shape=(None, None), dtype=int32)
I

In [90]:
# Please have a look at tf_transformers/extra/*.py for reference values

input_ids  = tf.constant([[1, 9, 10, 11, 23], 
                         [1, 22, 234, 432, 2349]])
input_mask = tf.ones_like(input_ids)
input_type_ids = tf.ones_like(input_ids)

inputs = {'input_ids': input_ids, 
          'input_mask': input_mask, 
          'input_type_ids': input_type_ids}

results_tf_transformers   = model_tf_transformers(inputs)
for k, r in results_tf_transformers.items():
    if isinstance(r, list):
        continue
    print(k, '-->', tf.reduce_sum(r), '-->', r.shape)

cls_output --> tf.Tensor(12.337944, shape=(), dtype=float32) --> (2, 768)
token_embeddings --> tf.Tensor(-193.53145, shape=(), dtype=float32) --> (2, 5, 768)


In [146]:
class NERModel(tf.keras.Model):

    def __init__(self, 
                 base_model,
                 slot_num_labels=None, 
                 dropout_prob=0.1, 
                 is_training=False):
        super().__init__(name="albert_ner")
        self.model = base_model
        self.dropout = tf.keras.layers.Dropout(dropout_prob)
        self.slot_classifier = tf.keras.layers.Dense(slot_num_labels,
                                     name="slot_classifier")
        self.is_training = is_training

    def call(self, inputs):
        outputs = self.model(inputs)
        
        for layer_iter, token_embeddings in enumerate(outputs['all_layer_token_embeddings']):
            sequence_output = token_embeddings
            sequence_output = self.dropout(sequence_output,
                                           training=self.is_training)
            slot_logits = self.slot_classifier(sequence_output)
            outputs['slot_logits_{}'.format(layer_iter+1)] = slot_logits
        return outputs
    
model = NERModel(model_tf_transformers, slot_num_labels=len(slot_map), is_training=True)

In [147]:
num_layers = 12
def loss_fn(results_dict, y_dict):
    
    all_losses = []
    labels = y_dict['token_labels']
    label_weights = y_dict['labels_mask']
    for layer_iter in range(num_layers):
        logits = results_dict['slot_logits_{}'.format(layer_iter+1)]
        per_example_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
        per_example_loss = per_example_loss * tf.cast(label_weights, per_example_loss.dtype)
        if label_weights is None:
            label_weights = tf.ones_like(labels)
        numerator   = tf.reduce_sum(per_example_loss)
        denominator = tf.cast(tf.reduce_sum(label_weights), numerator.dtype)
        denominator = tf.reduce_sum(label_weights)
        loss = tf.math.divide_no_nan(numerator, tf.cast(denominator, numerator.dtype))
        all_losses.append(loss)
    return tf.reduce_mean(all_losses)

In [ ]:
import os

STEPS_PER_CALL = 100       # No of steps to perform inside each call to train function
training_loss = tf.keras.metrics.Mean('training_loss', dtype=tf.float32) # We store loss here and reset after every global steps




# Optimizer
learning_rate = 2e-5
num_train_examples = 13000
steps_per_epoch = int(num_train_examples/ batch_size)
warmup_steps = int(EPOCHS * num_train_examples * 0.1 / batch_size)
optimizer_type = 'adamw'
optimizer = optimization.create_optimizer(learning_rate,
                                                steps_per_epoch * EPOCHS,
                                                warmup_steps,
                                                optimizer_type)
@tf.function
def train(iterator):
    """The step function for one training step"""
    def train_step(batch_inputs, batch_labels):
        """The computation to run on each TPU device."""
        with tf.GradientTape() as tape:
            model_outputs = model(batch_inputs)
            loss = loss_fn(model_outputs, batch_labels) 
            
        grads = tape.gradient(loss, model.variables)
        optimizer.apply_gradients(zip(grads, model.variables))
        # training_loss.update_state(loss * strategy.num_replicas_in_sync)
        training_loss.update_state(loss)

    for _ in tf.range(tf.convert_to_tensor(STEPS_PER_CALL)):
        batch_inputs, batch_labels = next(iterator)
        loss = train_step(batch_inputs, batch_labels)
        
        
# Make dataset iterator

GLOBAL_STEP    =  EPOCHS * (num_train_examples  // (batch_size * STEPS_PER_CALL))
logging.info("Global Steps {}".format(GLOBAL_STEP))
iter_dataset = iter(dataset)

checkpoint_dir = '/mnt/home/PRE_MODELS/tf_transformers_models/albert_nlu_full_layerwise'
if os.path.exists(checkpoint_dir):
    raise(FileExistsError)
    
ckpt = tf.train.Checkpoint(step=tf.Variable(1), model = model)
manager = tf.train.CheckpointManager(ckpt, checkpoint_dir, max_to_keep=5)

import time
for step_iter in range(GLOBAL_STEP):
    start_time = time.time()
    train(iter_dataset)
    ckpt.step.assign_add(1)
    end_time = time.time()
    logging.info("Global step {}, time {} seconds, loss {}".format(step_iter, end_time-start_time, training_loss.result()))
    
    training_loss.reset_states()
    
    print()
    print("--------------------------------------")
    
    if int(ckpt.step) % 100 == 0:
        save_path = manager.save()
        print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
save_path = manager.save()
print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))

print("Saving base model separately for using it in other tasks")
checkpoint_base_model_dir = checkpoint_dir + '_base'
if os.path.exists(checkpoint_base_model_dir):
    raise(FileExistsError)
ckpt    = tf.train.Checkpoint(model=model_layer)
manager = tf.train.CheckpointManager(ckpt, checkpoint_base_model_dir, max_to_keep=1)
save_path  = manager.save()
print("Saved at {}".format(save_path))

In [193]:
# for index, row in df_valid.iterrows():
#     print(row['words'], '-->', row['word_labels'])

Estelle should be on my spring playlist --> B-artist O O O B-playlist_owner B-playlist O
Can jovino santos neto 's album get added to my Confidence Boost playlist? --> O B-artist I-artist I-artist O B-music_item O O O B-playlist_owner B-playlist I-playlist O
Add to ila's playlist A Mi Manera Recopilatorio the name my heart stood still --> O O B-playlist_owner O B-playlist I-playlist I-playlist I-playlist O O B-entity_name I-entity_name I-entity_name I-entity_name
add sam sparro to my playlist called Beach Vibes --> O B-artist I-artist O B-playlist_owner O O B-playlist I-playlist
Please put this song onto my Urban Hits playlist. --> O O O B-music_item O B-playlist_owner B-playlist I-playlist O
add Hey, Johnnie Cope, Are Ye Waking Yet to year in metal 2016 playlist --> O B-entity_name I-entity_name I-entity_name I-entity_name I-entity_name I-entity_name I-entity_name O B-playlist I-playlist I-playlist I-playlist O
Add this track to my Rock Hard playlist --> O O B-music_item O B-playlist_

What time is The Hotline playing at the movie theatre --> O O O B-movie_name I-movie_name O O O B-object_location_type I-object_location_type
fimd films around here --> O B-movie_type B-spatial_relation I-spatial_relation
Show me movie time for I Am Sorry at my movie house --> O O O O O B-movie_name I-movie_name I-movie_name O O B-object_location_type I-object_location_type
Find the movie schedules for movies close by --> O O B-object_type I-object_type O B-movie_type B-spatial_relation I-spatial_relation
find a movie theatre with Life and Death --> O O B-object_location_type I-object_location_type O B-movie_name I-movie_name I-movie_name
What time is Doc Savage: The Man of Bronze playing --> O O O B-movie_name I-movie_name I-movie_name I-movie_name I-movie_name I-movie_name O
I want to see the movie schedules for animated movies around here . --> O O O O O B-object_type I-object_type O B-movie_type I-movie_type B-spatial_relation I-spatial_relation O
Find a movie schedule . --> O O B-

In [153]:
sentence = "Book a table for two at Le Ritz for Friday night!"
sentence = "Add to ila's playlist A Mi Manera Recopilatorio the name my heart stood still"

In [171]:
sentence = "Add to ila's playlist A Mi Manera Recopilatorio the name my heart stood still"

O_S = [] # original sentence
O_L = [] # original labels
P_L = [] # predicted labels

P_L_dict = {layer_iter+1: [] for layer_iter in range(num_layers)}

for row_idx, row in df_valid.iterrows():

    sentence = row['words']
    words_tokens = sentence.split()
    original_labels = row['word_labels'].split()
    subwords = tokenizer.tokenize(sentence)
    
    O_S.append(words_tokens)
    O_L.append(original_labels)

    # Convert text into main_words (list of list of subwords per word)
    main_words  = []
    temp_tokens = []
    for tok in subwords:
        if tok == SPECIAL_PIECE:
            if temp_tokens:
                main_words.append(temp_tokens)
                temp_tokens = []
            main_words.append([tok])

        else:
            if tok.startswith(SPECIAL_PIECE):
                if temp_tokens:
                    main_words.append(temp_tokens)
                    temp_tokens = []
                temp_tokens.append(tok)
            else:
                temp_tokens.append(tok)

    if temp_tokens:
        main_words.append(temp_tokens)

    special_counter = 0
    aligned_words  = []
    aligned_labels = []
    for index,w in enumerate(main_words):
        if len(w) == 1 and w[0] == SPECIAL_PIECE:
            special_counter += 1
            aligned_words.append(w[0])
        else:
            pos = index - special_counter
            aligned_words.append(words_tokens[pos])

    flat_tokens = []
    # The first word of the subword token is assigned entity
    # other tokens will be add PAD labels (we will mask it while training)
    for (aw, mw) in zip(aligned_words, main_words):
        temp_w = []
        for w in mw:
            temp_w.append(w)
        flat_tokens.extend(temp_w)
    flat_tokens = ['[CLS]'] + flat_tokens + ['[SEP]']

    input_ids  = tokenizer.convert_tokens_to_ids(flat_tokens)
    input_type_ids = [0] * len(input_ids)
    input_mask = [1] * len(input_ids)


    input_data = {}
    input_data['input_ids'] = tf.reshape(input_ids, (1, -1))
    input_data['input_type_ids'] = tf.reshape(input_type_ids, (1, -1))
    input_data['input_mask'] = tf.reshape(input_mask, (1, -1))

    results = model(input_data)
    
    for layer_iter in range(num_layers):
        slot_logits = results['slot_logits_{}'.format(layer_iter+1)]
        slot_ids = slot_logits.numpy().argmax(axis=-1)[0, 1:-1]

        # Extract only words that we want
        subword_counter = -1
        predicted_ids = []
        for sub_word_list in main_words:
            if len(sub_word_list) == 1 and sub_word_list[0] == SPECIAL_PIECE:
                subword_counter += 1
                continue
            else:
                predicted_ids.append(slot_ids[subword_counter+1])
                subword_counter += len(sub_word_list)

        predicted_labels = [slot_map_reverse[idx] for idx in predicted_ids]
        P_L_dict[layer_iter+1].append(predicted_labels)

        assert(len(words_tokens) == len(original_labels) == len(predicted_labels))
    print(row_idx)

In [ ]:
for layer_iter in range(num_layers):
    exact_match = []
    P_L = P_L_dict[layer_iter+1]
    for i in range(len(O_L)):
        if O_L[i] == P_L[i]:
            exact_match.append(i)
    print("Layer {} exact match {} / 700".format(layer_iter, len(exact_match)))

# Layer 0 exact match 196 / 700
# Layer 1 exact match 380 / 700
# Layer 2 exact match 470 / 700
# Layer 3 exact match 515 / 700
# Layer 4 exact match 540 / 700
# Layer 5 exact match 570 / 700
# Layer 6 exact match 567 / 700
# Layer 7 exact match 577 / 700
# Layer 8 exact match 585 / 700
# Layer 9 exact match 582 / 700
# Layer 10 exact match 577 / 700
# Layer 11 exact match 576 / 700

In [168]:
from sklearn_crfsuite import metrics


In [ ]:
layer_classification_report = {}
label_classes = [slot_map_reverse[i] for i in range(len(slot_map))]
for layer_iter in range(num_layers):
    P_L = P_L_dict[layer_iter+1]
    report  = metrics.flat_classification_report(O_L, 
                                          P_L, 
                                          labels=label_classes, digits=3)
    layer_classification_report[layer_iter+1] = report



In [ ]:
# Convert the model to tflite

config['num_hidden_layers'] = 5
model_layer_tflite = AlbertEncoder(config=config,
                      name='albert',
                      mask_mode=config['mask_mode'],
                      is_training=False, 
                      use_dropout=False,
                      batch_size = 1,
                      sequence_length = 40,
                      model_dir='/mnt/home/PRE_MODELS/tf_transformers_models/albert_nlu_full_layerwise_base'
                      )
# model_dir = None, because we have not initialized the model with proper variable values
model_tf_transformers_tflite = model_layer_tflite.get_and_load_model()


class NERModel(tf.keras.Model):

    def __init__(self, 
                 base_model,
                 slot_num_labels=None, 
                 dropout_prob=0.1, 
                 is_training=False):
        super().__init__(name="albert_ner")
        self.model = base_model
        self.dropout = tf.keras.layers.Dropout(dropout_prob)
        self.slot_classifier = tf.keras.layers.Dense(slot_num_labels,
                                     name="slot_classifier")
        self.is_training = is_training

    def call(self, inputs):
        outputs = self.model(inputs)
        
        sequence_output = outputs['token_embeddings']
        sequence_output = self.dropout(sequence_output,
                                       training=self.is_training)
        slot_logits = self.slot_classifier(sequence_output)
        results = {}
        results['slot_logits'] = slot_logits
        return results
    

# Ner model
ner_model_tflite = NERModel(model_tf_transformers_tflite, slot_num_labels=len(slot_map), is_training=False)

dummy_inputs = {}
dummy_inputs['input_ids'] = tf.ones(shape=(1, 40), dtype=tf.int32)
dummy_inputs['input_type_ids'] = tf.zeros_like(dummy_inputs['input_ids'])
dummy_inputs['input_mask'] = tf.ones_like(dummy_inputs['input_ids'])


sample_call = ner_model_tflite(dummy_inputs)
# Always make a call to initialize the variables


model_dir = '/mnt/home/PRE_MODELS/tf_transformers_models/albert_nlu_full_layerwise'
ckpt     = tf.train.Checkpoint(model=ner_model_tflite)
manager  = tf.train.CheckpointManager(ckpt, model_dir, max_to_keep=1)
status = ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    status.assert_existing_objects_matched()
    logging.info("Succesfully loaded from {}".format(manager.latest_checkpoint))
else:
    logging.info("No model checkpoint found at directory {}".format(model_dir))
    

In [ ]:
ner_model_tflite.save("/mnt/home/PRE_MODELS/tf_transformers_models/albert_nlu_layer5", save_format='tf')

In [ ]:
# Convert the model
model_dir = '/mnt/home/PRE_MODELS/tf_transformers_models/albert_nlu_layer5'
converter = tf.lite.TFLiteConverter.from_saved_model(model_dir) # path to the SavedModel directory
# converter.experimental_new_converter = False
converter.experimental_new_converter = True

tflite_model = converter.convert()

open("/mnt/home/PRE_MODELS/tf_transformers_models/albert_nlu_layer5.tflite", "wb").write(tflite_model)
